# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

<Client: 'inproc://10.1.0.4/8249/1' processes=1 threads=4, memory=2.00 GB>

## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
2000-01-01      int64  object  float64  float64
2000-01-02        ...     ...      ...      ...
...               ...     ...      ...      ...
2000-01-30        ...     ...      ...      ...
2000-01-31        ...     ...      ...      ...
Dask Name: make-timeseries, 30 tasks

In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
                int64  object  float64  float64
                  ...     ...      ...      ...
...               ...     ...      ...      ...
                  ...     ...      ...      ...
                  ...     ...      ...      ...
Dask Name: read-csv, 30 tasks

In [6]:
df.head()

id    name         x         y
0  1002  Yvonne -0.901267 -0.321609
1  1006     Ray -0.052166  0.273730
2   945  Xavier  0.997803 -0.269708
3  1011   Jerry -0.460786 -0.145117
4   996  Victor -0.316761  0.144087

Let's look at some statistics on the data

In [7]:
df.describe().compute()

id             x             y
count  2.592000e+06  2.592000e+06  2.592000e+06
mean   9.999925e+02  5.984082e-05 -3.959203e-04
std    3.163229e+01  5.773579e-01  5.771473e-01
min    8.360000e+02 -9.999996e-01 -9.999995e-01
25%    9.790000e+02 -4.940236e-01 -4.940382e-01
50%    1.000000e+03  9.052080e-03  5.260291e-03
75%    1.021000e+03  5.073613e-01  5.068001e-01
max    1.170000e+03  9.999987e-01  9.999992e-01

# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
1006,Zelda,0.8384785476958414
1024,Dan,-0.6825194095149374
987,Norbert,0.5630966010426157
1022,Xavier,0.28416420115214946
1054,Ray,-0.5327250043885807
952,Patricia,0.3491438028585492
1043,Sarah,-0.7864618492063187
1043,Ingrid,-0.44253483015913386
987,Michael,0.6323467455980205


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

id    name         x         y
0  1002  Yvonne -0.901267 -0.321609
1  1006     Ray -0.052166  0.273730
2   945  Xavier  0.997803 -0.269708
3  1011   Jerry -0.460786 -0.145117
4   996  Victor -0.316761  0.144087

Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((subgraph_callable, (<function pandas_read_text at 0x7f52743a8430>, <function _make_parser_function.<locals>.parser_f at 0x7f52764f09d0>, (<function read_block_from_file at 0x7f526d35fc10>, <OpenFile '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-07.csv'>, 0, 64000000, b'\n'), b'id,name,x,y\n', {}, {'id': dtype('int64'), 'name': dtype('O'), 'x': dtype('float64'), 'y': dtype('float64')}, ['id', 'name', 'x', 'y'], False, False, None), 'read-csv-ae7816158eae912ecd6c13f9ed5af616'))
kwargs:    {}
Exception: ValueError('Length mismatch: Expected axis has 3 elements, new values have 4 elements')



ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

id    name         x         y
0  1002  Yvonne -0.901267 -0.321609
1  1006     Ray -0.052166  0.273730
2   945  Xavier  0.997803 -0.269708
3  1011   Jerry -0.460786 -0.145117
4   996  Victor -0.316761  0.144087

In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

Empty DataFrame
Columns: [id, name, x, y]
Index: []

In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

id     name         x   y
0  1006    Zelda  0.838479 NaN
1  1024      Dan -0.682519 NaN
2   987  Norbert  0.563097 NaN
3  1022   Xavier  0.284164 NaN
4  1054      Ray -0.532725 NaN

# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-3b636e97-13ec-4239-bb2f-26891b6827f5')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

id      name         x         y
0      1002    Yvonne -0.901267 -0.321609
1      1006       Ray -0.052166  0.273730
2       945    Xavier  0.997803 -0.269708
3      1011     Jerry -0.460786 -0.145117
4       996    Victor -0.316761  0.144087
...     ...       ...       ...       ...
86395   981       Tim -0.619367  0.938098
86396   969  Patricia -0.332800 -0.815151
86397   999    Yvonne -0.081302  0.221984
86398  1030       Dan -0.727693  0.161516
86399  1027     Frank -0.096807 -0.547052

[86400 rows x 4 columns]

Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-17.csv',
 'data/2000-01-10.csv',
 'data/2000-01-19.csv',
 'data/2000-01-06.csv',
 'data/2000-01-29.csv',
 'data/2000-01-27.csv',
 'data/2000-01-18.csv',
 'data/2000-01-08.csv',
 'data/2000-01-30.csv',
 'data/2000-01-02.csv',
 'data/2000-01-14.csv',
 'data/2000-01-01.csv',
 'data/2000-01-21.csv',
 'data/2000-01-13.csv',
 'data/2000-01-05.csv',
 'data/2000-01-12.csv',
 'data/2000-01-24.csv',
 'data/2000-01-25.csv',
 'data/2000-01-09.csv',
 'data/2000-01-23.csv',
 'data/2000-01-16.csv',
 'data/2000-01-15.csv',
 'data/2000-01-22.csv',
 'data/2000-01-11.csv',
 'data/2000-01-28.csv',
 'data/2000-01-03.csv',
 'data/2000-01-26.csv',
 'data/2000-01-07.csv',
 'data/2000-01-04.csv',
 'data/2000-01-20.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-e657a29b-de45-4f90-b371-0970f4ec6a06'),
 Delayed('read_data-a70f98ae-61ff-4c15-aac8-8f881248e1e7'),
 Delayed('read_data-8c77e43f-8332-4d75-b5dc-17018e7c49ad'),
 Delayed('read_data-4982d72b-3033-480b-b53e-02608023d3b1'),
 Delayed('read_data-4fb00e8e-b169-4938-9f0d-d745e829f376'),
 Delayed('read_data-e146aa62-f558-45dc-bf8d-f4d5a96f0fe4'),
 Delayed('read_data-2bf1d07e-cfac-44b7-995d-c46d9b52192a'),
 Delayed('read_data-1b4d315c-13d5-4074-adb6-c72d7461d675'),
 Delayed('read_data-3a9936e5-942f-4545-a646-c9e2803fa5f3'),
 Delayed('read_data-0530bd02-4f57-49a3-8b3a-12ad059bf683'),
 Delayed('read_data-06308357-9029-47e0-9639-d838ba338c93'),
 Delayed('read_data-f08776ff-48e0-4fc4-83bb-750261952b67'),
 Delayed('read_data-f094f23e-7e8c-4d26-abdc-db306690ba21'),
 Delayed('read_data-281067b6-5436-4f9c-9db1-c18337d4759c'),
 Delayed('read_data-3b2ddd63-cc9b-49d7-8a87-5fecc326d7d4'),
 Delayed('read_data-60c1137a-a049-43d2-b8e3-a9c18a0f92fd'),
 Delayed('read_data-32c41110-3b97-4e7e-b

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.from_delayed). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
                int64  object  float64  float64
                  ...     ...      ...      ...
...               ...     ...      ...      ...
                  ...     ...      ...      ...
                  ...     ...      ...      ...
Dask Name: from-delayed, 60 tasks

Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

id      name         x         y
0   988     Jerry  0.417713  0.252505
1   937  Patricia -0.919289  0.681353
2   975     Edith -0.769674 -0.971839
3  1005    Oliver  0.829058 -0.634989
4  1035     Kevin  0.257129  0.551410

In [26]:
df.describe().compute()

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


id             x             y
count  2.505600e+06  2.505600e+06  2.419200e+06
mean   9.999997e+02  6.779305e-05 -5.031979e-04
std    3.163667e+01  5.773225e-01  5.770875e-01
min    8.360000e+02 -9.999982e-01 -9.999995e-01
25%    9.790000e+02 -4.940236e-01 -4.940382e-01
50%    1.000000e+03  9.052080e-03  5.260291e-03
75%    1.021000e+03  5.073613e-01  5.068001e-01
max    1.170000e+03  9.999987e-01  9.999992e-01

## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.